In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm

In [16]:
def extract(file):
    ang = np.linspace(-180*0.999, 180*0.999, 360) * np.pi/180
    y = []
    data = np.array(pd.read_csv(file))
    x = data[:,:5].reshape(-1,5)
    for i in range(len(x)):
        y_i = list(data[i,5][1:-2].split(','))
        y.append(y_i)
    y = np.array(y).astype(float)
    return x,y

x0,y0 = extract(f'Phase_NN_data_jup0.csv')
x1,y1 = extract(f'Phase_NN_data_jup1.csv')
x2,y2 = extract(f'Phase_NN_data_jup2.csv')
x = np.vstack((x0,x1,x2))
y = np.vstack((y0,y1,y2))
print(x[0])
print(y.shape)

    

[12.151192399859552 432.2319848111658 2.0809360292189343
 2.2461137115316888e-06 4.593880688016641]
(52400, 360)


In [ ]:
# random sin data



In [83]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        ls = [5,360,360,360,360,360,360,360,360]
        self.l1 = nn.Linear(ls[0],ls[1]) 
        self.l2 = nn.Linear(ls[1],ls[2]) 
        self.l3 = nn.Linear(ls[2],ls[3])
        self.l4 = nn.Linear(ls[3],ls[4])
        self.l5 = nn.Linear(ls[4],ls[5])
        self.l6 = nn.Linear(ls[5],ls[6])
        self.l7 = nn.Linear(ls[6],ls[7])
        self.l8 = nn.Linear(ls[7],ls[8])

    def forward(self, x):
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        x = F.relu(self.l5(x))
        x = F.relu(self.l6(x))
        x = F.relu(self.l7(x))
        x = self.l8(x)
        return x


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = Net().to(device)        
# Hyper-parameters 
test_pct = 0.1
EPOCHS = 500
BATCH_SIZE = 1000
learning_rate = 0.00001

optimizer = optim.Adam(net.parameters(), lr=learning_rate)

#optimizer = optim.SGD(net.parameters(), lr=learning_rate)
loss_function = nn.MSELoss()
#loss_function = nn.L1Loss()
val_size = int(len(x) * test_pct)
### add in randomization of index
shuf = np.random.permutation(len(x))
x = x[shuf].astype(np.float32)
y = y[shuf].astype(np.float32)

X = torch.from_numpy(x)
Y = torch.from_numpy(np.log(y))

trainX = X[:-val_size].to(device)   
trainY = Y[:-val_size].to(device)   
testX = X[-val_size:].to(device)   
testY = Y[-val_size:].to(device)  

# Train the model
eplos = []
for epoch in tqdm(range(EPOCHS)):
    for i in range(0, len(trainX), BATCH_SIZE): # from 0, to the len of x, stepping BATCH_SIZE at a time. [:50] ..for now just to dev

        batch_X = trainX[i:i+BATCH_SIZE]
        batch_y = trainY[i:i+BATCH_SIZE]

        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step() 
        net.zero_grad()
    eplos.append([epoch, loss])
print(f"Epoch: {epoch}. Loss: {loss}")

eplos = torch.tensor(eplos).to('cpu')
#eplos = np.array(eplos.to('cpu'))
plt.plot(eplos[:,0],eplos[:,1])
plt.semilogy()

 51%|████████████████████████████████████████▉                                       | 256/500 [06:33<06:14,  1.54s/it]


KeyboardInterrupt: 

In [158]:
print(type(x[1,0]))


<class 'float'>


In [60]:
X = torch.tensor([[1],[2],[3],[4]], dtype=torch.float32)
Y = torch.tensor([[2],[4],[6],[8]], dtype=torch.float32)

X_test = torch.tensor([5], dtype=torch.float32)

n_samples, n_features = X.shape
print(n_samples, n_features)

input_size = n_features
output_size = n_features

#model = nn.Linear(input_size, output_size)

class LinearRegression(nn.Module):
    
    def __init__(self, input_dim, output_dim):
        super(LinearRegression, self).__init__()
        #define layers
        self.lin = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        return self.lin(x)
    
model = LinearRegression(input_size, output_size)

print(f'Prediction before training: f(5) = {model(X_test).item():.3f}')

# Training 
learning_rate = 0.01
n_iters = 100

loss = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for epoch in range(n_iters):
    # prediction = forward pass
    y_pred = model(X)
    
    # loss
    l = loss(Y, y_pred)
             
    # gradients = backward pass
    l.backward()
             
    # update weights
    optimizer.step()
             
    # zero gradients
    optimizer.zero_grad()
    
    if epoch % 10 == 0:
        [w,b] = model.parameters()
        print(f'epoch {epoch+1}: w = {w[0][0].item():.3f}, loss = {l:.8f}')

print(f'Prediction after training: f(5) = {model(X_test).item():.3f}')


4 1
Prediction before training: f(5) = -2.501
epoch 1: w = 0.051, loss = 51.08811951
epoch 11: w = 1.688, loss = 1.32177675
epoch 21: w = 1.951, loss = 0.03420247
epoch 31: w = 1.994, loss = 0.00088962
epoch 41: w = 2.000, loss = 0.00002747
epoch 51: w = 2.002, loss = 0.00000490
epoch 61: w = 2.002, loss = 0.00000408
epoch 71: w = 2.002, loss = 0.00000383
epoch 81: w = 2.002, loss = 0.00000360
epoch 91: w = 2.002, loss = 0.00000339
Prediction after training: f(5) = 10.003
